# Intruder Detection using Deep Learning version 2 
Referenced from :
https://github.com/ageitgey/face_recognition/blob/master/examples/facerec_from_webcam.py

### Here we use face_recognition python module. This module runs upon dlib a c++ based library.
#### It consists of 4 steps.
* Step-1 : Encode the picture using the HOG algorithm to create a simplified version of the image. 
* Step-2 : Finding the main landmarks in the face like nose, mouth and ears.
* Step-3 : Encoding Faces. Here we use a pre-trained Convolution Neural Network developed by OpenFace.
* Step-4 : Finding the person’s name from the encoding.Compare which person has the closest measurements to our face’s measurements.

In [1]:
import glob
import face_recognition
import cv2
import numpy as np
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from datetime import datetime

### Data Set Understanding 

In [2]:
paths = glob.glob('D:\\project\\data\\*')
names = []
images = []
image_encodings = []
image_names = []
count_img = 0
for i in paths:
    images.append(face_recognition.load_image_file(i))
    image_encodings.append(face_recognition.face_encodings(images[count_img])[0])
    image_names.append(i.split('\\')[-1].split('.')[0])
    count_img+=1
    print(image_names)

['bill gates']
['bill gates', 'elon musk']
['bill gates', 'elon musk', 'jeff bezos']
['bill gates', 'elon musk', 'jeff bezos', 'Milind']
['bill gates', 'elon musk', 'jeff bezos', 'Milind', 'obama']


In [3]:
server = smtplib.SMTP('smtp.gmail.com',587)
server.starttls()
username = 'abc@gmail.com'
passwd = 'pass'
server.login(username,passwd)

(235, b'2.7.0 Accepted')

## Data encoding, Training and detecting

In [5]:
count = 0
cap = cv2.VideoCapture(0)
while True:
    ret,frame = cap.read()
    gray = frame[:, :, ::-1]
    face_locations = face_recognition.face_locations(gray)
    face_encodings = face_recognition.face_encodings(gray, face_locations)
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(image_encodings, face_encoding)
        name = 'Unknown'
        face_distances = face_recognition.face_distance(image_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = image_names[best_match_index]
        if(name=='Unknown'):
            cv2.imwrite('D:\\project\\intruders\\intru-{}.jpg'.format(count),frame)
#             server = smtplib.SMTP('smtp.gmail.com',587)
#             server.starttls()
#             username = 'kaluramgates@gmail.com'
#             passwd = 'K@luramGates321'
#             server.login(username,passwd)
            msg= MIMEMultipart()
            msg['from'] = username
            msg['to'] = username
            msg['subject'] = "[Urgent]Intruder Detected"
            text = "Check these sample unknown person images, if some thing *fishy!!* check intruder folder immediately"
            msg.attach(MIMEText(text))
            filename = "D:\project\intruders\intru-{}.jpg".format(count)
            f=open(filename,'rb')
            part = MIMEApplication(f.read())
            f.close()
            part.add_header('content-Disposition','attachment',filename=filename)
            msg.attach(part)
            server.sendmail(username,username,msg.as_string())
            count+=1
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 3)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
    cv2.imshow("output",frame)
    if(cv2.waitKey(1)==ord('q')):
        break
cap.release()
cv2.destroyAllWindows()